In [7]:
import numpy as np
import pandas as pd
from river import evaluate
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sail.transformers.river.preprocessing import StandardScaler
from sail.models.torch.rnn import RNNRegressor

#### Load Data


In [8]:
X = pd.read_csv("../../datasets/HDWF2.csv")

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [9]:
learner_gru = RNNRegressor(
    input_units=12,
    output_units=1,
    hidden_units=100,
    n_hidden_layers=3,
    lr=0.001,
    cell_type="GRU",
    verbose=0,
)

#### Create SAIL Pipeline


In [10]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("regressor", learner_gru),
]
sail_pipeline = SAILPipeline(
    steps=steps, scoring=["R2"], verbosity_level=1, verbosity_interval=100
)

#### Train Test Split


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

#### Start Incremental Training


In [12]:
y_preds = []
y_true = []
batch_size = 10

for start in range(0, 5000, batch_size):
    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_pipeline.partial_fit(X, y)


    
>> Epoch: 1 | Samples Seen: 0 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.031s, Batch Size=10, P_Score=0]            

    
>> Epoch: 100 | Samples Seen: 990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.006s, Batch Size=10, P_Score=-.28]            

    
>> Epoch: 200 | Samples Seen: 1990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.007s, Batch Size=10, P_Score=0.196]            

    
>> Epoch: 300 | Samples Seen: 2990 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.012s, Batch Size=10, P_Scor

#### Save SAIL Pipeline


In [13]:
sail_pipeline.save(".")

[2023-12-15 12:25:46:456] - INFO - SAIL (PyTorch) - Model saved successfully.


'./sail_pipeline'

#### Load SAIL Pipeline


In [14]:
sail_new_pipeline = SAILPipeline.load(".")

SAIL Pipeline Progressive Score: 100%========== [Points: 4990/4990, Elapsed:0.0094s, Metric=R2, Batch Size=4990, P_Score=0.507]


In [18]:
y_preds = []
y_true = []
batch_size = 10

for start in range(5001, 10768, batch_size):
    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_new_pipeline.partial_fit(X, y)

   1655      112.1236  0.0013
   1656      476.6375  0.0015
   1657      131.2363  0.0012
   1658       91.6342  0.0017
   1659      217.1892  0.0014
   1660      174.4440  0.0013
   1661      179.4757  0.0012
   1662      403.8756  0.0013
   1663      143.5877  0.0014
   1664      109.5449  0.0012
   1665      256.3188  0.0012
   1666      391.0991  0.0012
   1667      172.7421  0.0012
   1668      147.9162  0.0014
   1669      140.9670  0.0013
   1670      275.6855  0.0015
   1671      233.9279  0.0011
   1672      186.4308  0.0012
   1673      179.0110  0.0012
   1674      334.0541  0.0014
   1675      543.9901  0.0012
   1676      293.3163  0.0012
   1677      222.9568  0.0010
   1678      275.5040  0.0013
   1679      113.3826  0.0011
   1680      501.8510  0.0012
   1681      219.6418  0.0013
   1682       90.7520  0.0011
   1683      193.4498  0.0013
   1684       79.8159  0.0015
   1685       82.2701  0.0012
   1686       94.0591  0.0012
   1687      249.4489  0.0013
   1688   


    
>> Epoch: 1700 | Samples Seen: 16984 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit [regressor]:  67%======     [Steps: 2/3, ETA: 00:00<00:00, Elapsed:0.003s, Batch Size=10]      

   1700      319.0164  0.0021


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.007s, Batch Size=10, P_Score=0.67]            

   1701      390.7328  0.0086
   1702      797.7980  0.0013
   1703      150.2974  0.0013
   1704      317.3651  0.0014
   1705      154.1953  0.0014
   1706      143.8605  0.0014
   1707       97.9356  0.0015
   1708      184.7766  0.0012
   1709      175.2373  0.0014
   1710      394.0027  0.0012
   1711      128.2332  0.0011
   1712      174.6492  0.0011
   1713      144.3876  0.0013
   1714      272.8502  0.0011
   1715      298.4694  0.0010
   1716      150.7125  0.0010
   1717      231.1597  0.0015
   1718      241.2277  0.0012
   1719      435.7589  0.0011
   1720      230.0741  0.0014
   1721      155.2570  0.0013
   1722      559.3129  0.0012
   1723      141.7574  0.0010
   1724      343.6591  0.0013
   1725      109.2231  0.0010
   1726      289.6775  0.0010
   1727      478.7103  0.0013
   1728      197.1194  0.0013
   1729      122.4275  0.0014
   1730      231.1898  0.0011
   1731      133.2758  0.0010
   1732       87.5538  0.0011
   1733      175.4768  0.0013


...

   2131      153.1482  0.0019
   2132      184.8892  0.0015
   2133      413.8307  0.0011
   2134      187.3310  0.0011
   2135      118.4869  0.0011
   2136      381.9012  0.0012
   2137       99.6688  0.0012
   2138      205.5524  0.0010
   2139      249.2246  0.0013
   2140      229.9596  0.0013
   2141      137.1512  0.0011
   2142      353.1047  0.0015
   2143      727.5931  0.0012
   2144       47.6910  0.0013
   2145      142.6480  0.0012
   2146      455.1124  0.0011
   2147       74.2676  0.0010
   2148      218.8747  0.0016
   2149      108.0622  0.0012
   2150      650.6390  0.0014
   2151      255.1933  0.0011
   2152      208.6157  0.0015
   2153      148.7298  0.0012
   2154      338.1505  0.0011
   2155      355.6426  0.0010
   2156      195.8746  0.0010
   2157      361.4687  0.0013
   2158      144.1465  0.0012
   2159       36.9469  0.0012
   2160      101.1099  0.0011
   2161      229.0975  0.0013
   2162      108.1024  0.0012
   2163      114.0581  0.0015
   2164   


    
>> Epoch: 2200 | Samples Seen: 21984 -------------------------------------------------------------------------------------
SAIL Pipeline Partial fit [regressor]:  67%======     [Steps: 2/3, ETA: 00:00<00:00, Elapsed:0.003s, Batch Size=10]      

   2200       39.6993  0.0022


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.007s, Batch Size=10, P_Score=0.689]            

   2201      307.9598  0.0025
   2202       19.0244  0.0016
   2203      131.7699  0.0012
   2204      136.4696  0.0013
   2205      152.9515  0.0013
   2206      164.2231  0.0019
   2207      312.5992  0.0010
   2208      318.6282  0.0014
   2209       97.0151  0.0014
   2210      233.9873  0.0011
   2211      292.4092  0.0012
   2212      222.3992  0.0011
   2213      127.0963  0.0014
   2214      461.0698  0.0013
   2215      122.9068  0.0014
   2216      193.4299  0.0012
   2217       94.9604  0.0011
   2218      133.9912  0.0012
   2219      164.3865  0.0013
   2220      256.3975  0.0013
   2221      302.2519  0.0011
   2222      259.1956  0.0011
   2223      390.1907  0.0013
   2224      331.7269  0.0015
   2225      223.8464  0.0010
   2226      181.6559  0.0012
   2227      319.6913  0.0013
   2228      506.9944  0.0012
   2229      192.4312  0.0011
   2230      467.9801  0.0012
   2231      459.4061  0.0015


#### Make Prediction on Hold out set


In [19]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, X_test.shape[0], batch_size):
    end = start + batch_size

    X = X_test.iloc[start:end]
    y = y_test.iloc[start:end]

    preds = sail_new_pipeline.predict(X)
    y_preds.extend(list(preds))
    y_true.extend(list(y))

#### Final Score


In [20]:
sail_new_pipeline.get_progressive_score

0.6893614406963264

In [21]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds}).head(100)
fig = px.line(df, y=["y_true", "y_preds"], title="")
fig.show()